# Assignment 8

In [1]:
from __future__ import print_function

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import numpy as np
from tqdm.notebook import trange, tqdm
import matplotlib.pyplot as plt

torch.__version__

'1.2.0'

## Load test data and add noises

In [2]:
class numpyDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = torch.from_numpy(data).float()
        self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index]
        
        if self.transform:
            x = self.transform(x)
        
        return x
    
    def __len__(self):
        return len(self.data)

In [3]:
def activateNoises(imagedata, std_dev):
    imagedata += np.random.normal(0, std_dev, imagedata.shape)
    
    imagedata = np.minimum(imagedata, 1)
    imagedata = np.maximum(imagedata, 0)
    
    return imagedata

In [4]:
noises = [0.1, 0.2, 0.3, 0.4]

transform       = transforms.Compose([
                                transforms.ToPILImage(),
                                transforms.Grayscale(num_output_channels=1),
                                transforms.ToTensor(),
                            ])

# for training
traindata       = np.load('data/train.npy')
traindataset    = numpyDataset(traindata, transform)
trainloader     = DataLoader(traindataset, batch_size=1, shuffle=False, num_workers=0)

In [5]:
print(activateNoises(traindata, 0.4))

[[[0.         0.6051661  0.28552568 ... 0.61486304 1.         0.52739227]
  [0.3795036  0.83197945 0.59639096 ... 0.5760525  1.         0.11514256]
  [0.9641071  0.62923276 0.8952763  ... 0.65875006 0.37057683 0.10328747]
  ...
  [0.34951097 0.29382202 0.30376965 ... 0.72488797 0.38328022 0.        ]
  [0.22617392 0.5620622  1.         ... 0.826604   0.         0.        ]
  [0.94241345 0.         0.22451225 ... 0.48591673 0.6662454  0.        ]]

 [[0.6511332  0.46176583 0.4163122  ... 0.63801706 0.09100156 1.        ]
  [0.16304618 1.         1.         ... 0.32725805 0.5554964  0.09809058]
  [0.521878   0.5736621  0.71491396 ... 1.         0.03369563 0.644455  ]
  ...
  [0.         0.18703593 0.         ... 0.3195654  0.7078311  0.18092014]
  [0.4951032  0.17110664 0.20109786 ... 0.7768712  0.12868667 0.5528101 ]
  [0.6664603  1.         0.4512232  ... 0.45217937 0.         0.        ]]

 [[0.35172832 0.2864823  0.5046197  ... 0.9937363  1.         0.43436792]
  [0.6694437  0.552626